## Packages 
* [tslearn](https://tslearn.readthedocs.io/en/stable/quickstart.html)

## References
* [OpenFace 2.0](https://github.com/TadasBaltrusaitis/OpenFace/wiki): Facial Behavior Analysis Toolkit Tadas Baltrušaitis, Amir Zadeh, Yao Chong Lim, and Louis-Philippe Morency, IEEE International Conference on Automatic Face and Gesture Recognition, 2018

In [2]:
%ls

A3.ipynb               GIFGIF_Download.ipynb  venv/
GIFGIF_Dataset/        utilities/


## Dataset creation
I was looking to run OpenFace on the gifs, but that is apparently not possible. It only works on videos or images
